In [1]:
%pip install faiss-cpu sentence-transformers dashscope langchain

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 9.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.7/437.7 kB 1.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 1.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 443.9/443.9 kB 8.4 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 3.4 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 5.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 9.6 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.8/45.8 kB 656.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.9/583.9 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: typing_extensions
    Found

In [3]:
%pip install -U langchain-community

Looking in indexes: https://mirrors.tencent.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 4.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 189.2 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import faiss
import numpy as np
import os
from sentence_transformers import SentenceTransformer
from dashscope import Generation
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

# ====== 系统配置 ======
class Config:
    # 文件路径
    LAWS_FILE = "laws.txt"                # 原始法律条文文件
    FAISS_INDEX = "legal_index.faiss"     # FAISS索引文件
    
    # 模型参数
    EMBEDDING_MODEL = "paraphrase-multilingual-MiniLM-L12-v2"  # 语义编码模型
    LLM_MODEL = "qwen-turbo"             # 大语言模型
    DASHSCOPE_API_KEY = "sk-7fb2aee47f5d4531855a7ac3412249fe"   # 阿里云API密钥
    
    # 处理参数
    CHUNK_SIZE = 200                     # 文本分割长度
    TOP_K = 3                            # 检索返回条款数

# ====== 文档处理模块 ======
class LawProcessor:
    @staticmethod
    def load_and_split():
        """加载并分割法律文档"""
        if not os.path.exists(Config.LAWS_FILE):
            raise FileNotFoundError(f"法律文件 {Config.LAWS_FILE} 不存在")

        # 加载文档
        loader = TextLoader(Config.LAWS_FILE, encoding="utf-8")
        documents = loader.load()
        
        # 分割配置
        text_splitter = CharacterTextSplitter(
            separator="\n",
            chunk_size=Config.CHUNK_SIZE,
            chunk_overlap=20,
            keep_separator=True
        )
        
        # 执行分割
        split_docs = text_splitter.split_documents(documents)
        
        # 清洗处理
        clauses = [doc.page_content.strip() 
                 for doc in split_docs 
                 if len(doc.page_content.strip()) > 10]  # 过滤短文本
        
        return clauses

# ====== 索引管理模块 ======
class IndexManager:
    def __init__(self, clauses):
        self.clauses = clauses
        self.model = SentenceTransformer(Config.EMBEDDING_MODEL)
        
    def build_index(self):
        """构建FAISS索引"""
        embeddings = self.model.encode(self.clauses)
        dimension = embeddings.shape[1]
        
        index = faiss.IndexFlatL2(dimension)
        index.add(embeddings.astype(np.float32))
        faiss.write_index(index, Config.FAISS_INDEX)
        return index
    
    @staticmethod
    def load_existing_index():
        """加载已有索引"""
        if os.path.exists(Config.FAISS_INDEX):
            return faiss.read_index(Config.FAISS_INDEX)
        return None

# ====== 核心业务逻辑 ======
class LegalComplianceChecker:
    def __init__(self):
        # 初始化组件
        self.clauses = LawProcessor.load_and_split()
        self.index = IndexManager.load_existing_index() or \
                    IndexManager(self.clauses).build_index()
        self.encoder = SentenceTransformer(Config.EMBEDDING_MODEL)
    
    def query(self, user_input):
        """处理用户查询全流程"""
        # 1. 语义检索
        query_embedding = self.encoder.encode([user_input])
        distances, indices = self.index.search(query_embedding.astype(np.float32), Config.TOP_K)
        
        # 2. 获取相关条款
        relevant_clauses = [self.clauses[idx] for idx in indices[0] if idx < len(self.clauses)]
        
        # 3. 构建Prompt
        prompt = f"""执行法律合规性审查：
        [相关法律依据]
        {chr(10).join(relevant_clauses)}
        
        [待审查政策]
        {user_input}
        
        请按以下格式输出：
        1. 合规性结论（合规/部分合规/不合规）
        2. 判断理由（不超过50字）
        3. 依据条款（列出条款编号）"""
        
        # 4. 调用大模型
        response = Generation.call(
            model=Config.LLM_MODEL,
            prompt=prompt,
            api_key=Config.DASHSCOPE_API_KEY,
            max_length=500,
            top_p=0.7
        )
        
        return response.output.text if response.status_code == 200 else "请求失败"

# ====== 使用示例 ======
if __name__ == "__main__":
    # 初始化系统
    checker = LegalComplianceChecker()
    
    # 测试案例
    test_case = "地方政府要求网约车平台必须安装指定厂商的监控设备"
    
    # 执行审查
    result = checker.query(test_case)
    
    # 格式化输出
    print("=== 政策审查报告 ===")
    print(f"输入政策：{test_case}")
    print("\n审查结果：")
    print(result)

/root/.pyenv/versions/3.11.1/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


=== 政策审查报告 ===
输入政策：地方政府要求网约车平台必须安装指定厂商的监控设备

审查结果：
1. 不合规  
2. 强制指定设备厂商，限制了市场竞争。  
3. 第四条（一）、第五条（三）
